# https://pypi.org/project/duckduckgo-search/

In [2]:
!python -m pip install --upgrade pip
!pip install duckduckgo_search --q
!pip install textblob --q
!pip install unidecode --q
!pip install unescape --q
"Done"

ERROR: Could not find a version that satisfies the requirement unescape (from versions: none)
ERROR: No matching distribution found for unescape


'Done'

In [31]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')     # jika kamu pakai lemmatizer
nltk.download('stopwords') 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amarf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\amarf\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amarf\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [3]:
import time
from duckduckgo_search import DDGS
import pandas as pd
import unidecode, re, textblob
from textblob import TextBlob
from textblob import Word
from tqdm import tqdm

"Done"

'Done'

In [5]:
medsos = "facebook instagram x linkedin youtube tiktok".split()
medsos = [ms+".com" for ms in medsos]
media = "kompas detik liputan6 tribunnews".split()
media = [ms+".com" for ms in media]
keywords = ['"sertifikasi" "halal"']

medsos, media, keywords

(['facebook.com',
  'instagram.com',
  'x.com',
  'linkedin.com',
  'youtube.com',
  'tiktok.com'],
 ['kompas.com', 'detik.com', 'liputan6.com', 'tribunnews.com'],
 ['"sertifikasi" "halal"'])

In [ ]:
tidur = 360 #delay 360 detik
fData = 'data/Data_Halal.csv'
fresume = "data/sudah_diQuery.txt"
resume = False
try:
    with open(fresume, "r") as file:
    sudahQuery = file.readlines()
    if len(sudahQuery) < len(media) + len(medsos):
        resume = True
        df = pd.read_csv(fData)
        for ms in medsos:
            for kw in keywords:
                qry = "{} site:{}".format(kw, ms)
                print(qry)
                if qry not in sudahQuery:
                    belumDapetData = True
                    hasil = None
                    while belumDapetData:
                        try:
                            hasil = DDGS().text(qry, max_results=100)
                        except:
                            pass
                        if not hasil:
                            time.sleep(tidur) 
                        else:
                            belumDapetData = False
                    
                    hasil = pd.DataFrame(hasil)
                    hasil['source'] = ms
                    df = pd.concat([df, hasil], axis=0)
                    df.to_csv(fData, index=False)
                    with open(fresume, "a") as file:
                        file.write(qry)
                    time.sleep(tidur) 
        for ms in media:
            qry = 'halal site:{}'.format(ms)
            print(qry)
            if qry not in sudahQuery:            
                belumDapetData = True
                    hasil = None
                    while belumDapetData:
                        try:
                            hasil = DDGS().text(qry, max_results=100)
                        except:
                            pass
                        if not hasil:
                            time.sleep(tidur) 
                        else:
                            belumDapetData = False
                hasil = pd.DataFrame(hasil)
                hasil['source'] = ms
                df = pd.concat([df, hasil], axis=0)
                df.to_csv(fData, index=False)
                with open(fresume, "a") as file:
                        file.write(qry)
                time.sleep(tidur)
        
except: #mulai dari awal
    result = []
    for ms in medsos:
        for kw in keywords:
            qry = "{} site:{}".format(kw, ms)
            print(qry)
            belumDapetData = True
            hasil = None
            while belumDapetData:
                try:
                    hasil = DDGS().text(qry, max_results=100)
                except:
                    pass
                if not hasil:
                    time.sleep(tidur) 
                else:
                    belumDapetData = False
                    for data in hasil:
                        hasil['source'] = ms
            results.extend(hasil)
            time.sleep(tidur)    
    for ms in media:
        qry = 'halal site:{}'.format(ms)
        print(qry)
        belumDapetData = True
        hasil = None
        while belumDapetData:
            try:
                hasil = DDGS().text(qry, max_results=100)
            except:
                pass
            if not hasil:
                time.sleep(tidur) 
            else:
                belumDapetData = False
                for data in hasil:
                    hasil['source'] = ms
        results.extend(hasil)
        time.sleep(tidur)
    hasil = pd.DataFrame(result)
    df.to_csv(fData, index=False)

In [12]:
import unidecode, re, textblob
from textblob import Word

try:
    f = 'stopwords_id.txt' # atau stopwords_en.txt
    df=open(f,"r",encoding="utf-8", errors='replace')
except:
    pass

stopWords = df.readlines();df.close()
stopWords = [t.strip() for t in stopWords]
print(stopWords[:5])
stopWords = set(stopWords)

['ada', 'adalah', 'adanya', 'adapun', 'agak']


In [32]:
#Preprocessing 
def txtPreprocessing(txt):
    T = txt.lower() 
    T = re.sub(r'[^\w]',' ',T) # remove symbols
    T = TextBlob(T).words # Tokenisasi, T sekarang List
    T = [Word(t).lemmatize() for t in T] # Lemma
    T = [t for t in T if str(t) not in stopWords] # Untuk visualisasi data
    return ' '.join(T)

In [39]:
df['clean'] = ''
for i, d in tqdm(df.iterrows()):
    post_bersih = txtPreprocessing(d.title + ' ' + d.body)
    df.iloc[i, 'clean'] = post_bersih

df.to_csv(fData, index=False) # Simpan hasil preProcessing
df.head()

160


['lppom mui bogor facebook lppom mui bogor indonesia 145 648 like 408 talking about this fanpage official lppom mui lppom mui lembaga sertifikasi halal lsh dunia terakreditasi sni iso iec',
 'lph halal nusantara facebook sertifikasi halal tidak perspektif agama sudut pandang bisnis berbicara marketing branding produk irham sertifikasi halal alat pemasaran ampuh membantu bisnis menarik pelanggan negeri internasional',
 'info sertifikasi halal indonesia facebook group medium silaturahmi pelaku usaha penyelia halal berbagi informasi terkait sertifikasi halal jaminan produk halal']